In [1]:
# The pycocoevalcap folder is constructed by Tsung-Yi Lin(https://github.com/tylin)



In [7]:
import json
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider


In [8]:
def construct_dataset(ground_truth, result):
    
    datasetGTS = {}
    datasetRES = {}
    image_id = 0
    datasetGTS['annotations'] = []
    datasetRES['annotations'] = []
    for truth in ground_truth:
        single_piece = {}
        single_piece['image_id'] = image_id
        single_piece['caption'] = truth
        datasetGTS['annotations'].append(single_piece)
        image_id+=1
    image_id = 0
    for res in result:
        single_piece = {}
        single_piece['image_id'] = image_id
        single_piece['caption'] = res
        datasetRES['annotations'].append(single_piece)
        image_id+=1
    return datasetGTS,datasetRES

In [5]:
class COCOEvalCap:
    def __init__(self,images,gts,res):
        self.evalImgs = []
        self.eval = {}
        self.imgToEval = {}
        self.params = {'image_id': images}
        self.gts = gts
        self.res = res

    def evaluate(self):
        imgIds = self.params['image_id']
        gts = self.gts
        res = self.res

        # =================================================
        # Set up scorers
        # =================================================
        print 'tokenization...'
        tokenizer = PTBTokenizer()
        gts  = tokenizer.tokenize(gts)
        res = tokenizer.tokenize(res)
        print(gts)
        print(res)#:{0: ['man is playing guitar']}

        # =================================================
        # Set up scorers
        # =================================================
        print 'setting up scorers...'
        scorers = [
            (Bleu(3), ["Bleu_1", "Bleu_2", "Bleu_3"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr")
        ]

        # =================================================
        # Compute scores
        # =================================================
        eval = {}
        for scorer, method in scorers:
            print 'computing %s score...'%(scorer.method())
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, imgIds, m)
                    print "%s: %0.3f"%(m, sc)
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, imgIds, method)
                print "%s: %0.3f"%(method, score)
        self.setEvalImgs()

    def setEval(self, score, method):
        self.eval[method] = score

    def setImgToEvalImgs(self, scores, imgIds, method):
        for imgId, score in zip(imgIds, scores):
            if not imgId in self.imgToEval:
                self.imgToEval[imgId] = {}
                self.imgToEval[imgId]["image_id"] = imgId
            self.imgToEval[imgId][method] = score

    def setEvalImgs(self):
        self.evalImgs = [eval for imgId, eval in self.imgToEval.items()]

def calculate_metrics(rng,datasetGTS,datasetRES):
    imgIds = rng
    gts = {}
    res = {}

    imgToAnnsGTS = {ann['image_id']: [] for ann in datasetGTS['annotations']}
    for ann in datasetGTS['annotations']:
        imgToAnnsGTS[ann['image_id']] += [ann]

    imgToAnnsRES = {ann['image_id']: [] for ann in datasetRES['annotations']}
    for ann in datasetRES['annotations']:
        imgToAnnsRES[ann['image_id']] += [ann]

    for imgId in imgIds:
        print imgId
        gts[imgId] = imgToAnnsGTS[imgId]
        res[imgId] = imgToAnnsRES[imgId]

    evalObj = COCOEvalCap(imgIds,gts,res)
    evalObj.evaluate()
    return evalObj.eval

if __name__ == '__main__':
    # input = list of resulted strings
    true_captions = ['The man is playing a guitar?','the dog, is playing a piano.']
    predict_captions = ['man, is playing guitar?', 'dog, is playing piano.']
    
    
    #print len(true_captions)
    rng = range(len(true_captions)) # number of images
    datasetGTS,datasetRES = construct_dataset(true_captions, predict_captions)
    """
    datasetGTS = {
        'annotations': [{'image_id': 0, 'caption': u'the man is playing a guitar'}
                        ]
        }
    datasetRES = {
        'annotations': [{'image_id': 0, 'caption': u'man is playing guitar'}
                        ]
        }
    """
    print datasetGTS, datasetRES
    print calculate_metrics(rng,datasetGTS,datasetRES)

{'annotations': [{'image_id': 0, 'caption': 'The man is playing a guitar?'}, {'image_id': 1, 'caption': 'the dog, is playing a piano.'}]} {'annotations': [{'image_id': 0, 'caption': 'man, is playing guitar?'}, {'image_id': 1, 'caption': 'dog, is playing piano.'}]}
0
1
tokenization...
{0: ['the man is playing a guitar'], 1: ['the dog is playing a piano']}
{0: ['man is playing guitar'], 1: ['dog is playing piano']}
setting up scorers...
computing Bleu score...
{'reflen': 12, 'guess': [8, 6, 4], 'testlen': 8, 'correct': [8, 4, 2]}
ratio: 0.666666666611
Bleu_1: 0.607
Bleu_2: 0.495
Bleu_3: 0.421
computing METEOR score...
METEOR: 0.408
computing Rouge score...
ROUGE_L: 0.772
computing CIDEr score...
CIDEr: 4.296
{'Bleu_3': 0.4205448710225832, 'CIDEr': 4.2958303524133692, 'Bleu_2': 0.4952302097490788, 'Bleu_1': 0.6065306595610012, 'ROUGE_L': 0.77215189873417722, 'METEOR': 0.4082646683951501}


In [6]:
with open('true.txt') as infile: 
    datasetGTS = json.load(infile)
infile.close()

with open('predict.txt') as infile: 
    datasetRES = json.load(infile)
infile.close()
rng = range(len(datasetRES['annotations'])) # number of images

#print datasetGTS, datasetRES
print calculate_metrics(rng,datasetGTS,datasetRES)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{0: ['unk unk campaigner who unk sought to bring attention to and unk the practice of female unk unk -lsb- end -rsb-'], 1: ['sweden officially unk state of palestine in attempt to help end decades of conflict with israel -lsb- end -rsb-'], 2: ['mother cleared of poisoning teenager daughter with unk unk by belgian doctor says case should be landmark for parents rights -lsb- end -rsb-'], 3: ['unk quinn on gamergate it s not about ethical journalism it s unk revenge porn by my angry ex -lsb- end -rsb-'], 4: ['uk government criticised over unk plans to stop rescuing drowning african migrants -lsb- end -rsb-'], 5: ['uk weather halloween set to be warmest on record unk temperatures may soon drop -lsb- end -rsb-'], 6: ['mass grave containing 150 unk sunni unk fighters discovered by iraqi officials -lsb- end -rsb-'], 7: ['pope francis unk for the poor does not make you a communist -lsb- end -rsb-'], 8: ['100 days of ukip nigel farage becomes prime minister in new channel 4 unk -lsb- end -rsb-'

{'reflen': 35825, 'guess': [28953, 26686, 24419], 'testlen': 28953, 'correct': [9613, 5077, 2445]}
ratio: 0.808178646197
Bleu_1: 0.262
Bleu_2: 0.198
Bleu_3: 0.146
computing METEOR score...
METEOR: 0.147
computing Rouge score...
ROUGE_L: 0.286
computing CIDEr score...
CIDEr: 0.179
{'Bleu_3': 0.1458590801528023, 'CIDEr': 0.17940505771965976, 'Bleu_2': 0.19822784051627293, 'Bleu_1': 0.2618698472830026, 'ROUGE_L': 0.28557616179443474, 'METEOR': 0.14654094886944757}
